In [1]:
import itertools
from treys import Card
from treys import Evaluator
from treys import Deck

In [2]:
def lget(l, index, default=None):
    try:
        return l[index]
    except IndexError:
        return default

In [5]:
def typeOfCard(card, cardsDesired):
    if not cardsDesired:
        return True
    
    return card in cardsDesired

In [6]:
def simulatePokerSolitaire(cardsDesired, keyWord):
    playerCards, communityCards, deck = [], [], Deck()
    
    if keyWord == "FirstFive":
        cardsDesired = []
        cards = deck.draw(5)
        for card in cards:
            playerCards.append(card)
    count = 0

    while len(playerCards) != 5:
        card = deck.draw(1)
        count += 1
        if typeOfCard(Card.print_pretty_card(card), Card.print_pretty_cards(cardsDesired)):
            playerCards.append(card)
        else:
            communityCards.append(card)
    
    if len(communityCards) < 5:
        numOfCardsNeeded = 5 - len(communityCards)
        cards = deck.draw(numOfCardsNeeded)
        for card in cards:
            communityCards.append(card)
            
    return playerCards, communityCards

In [7]:
def findBestCombination(totalCombinations, highestScore, evaluator):
    for combination in totalCombinations:
        if evaluator.evaluate(list(combination), []) == highestScore:
            return list(combination)
    
    return None

In [8]:
def evaluateBetterHand(playerCards, communityCards):
    evaluator = Evaluator()
    playerScore = evaluator.evaluate(playerCards, [])
    totalCombinations = [combinationCommunityCards for combinationCommunityCards in itertools.combinations(communityCards, 5)]
    highestScore = min(evaluator.evaluate(list(combination), []) for combination in totalCombinations)
    bestCommunityCombination = findBestCombination(totalCombinations, highestScore, evaluator)
    
    print(playerScore, highestScore)
    return highestScore >= playerScore, bestCommunityCombination

In [9]:
def playPoker(enterHand, keyWord):
    numericCardHand = [((CHAR_RANK_TO_INT_RANK[lget(card, 0)] << 8) + 
                        (CHAR_SUIT_TO_INT_SUIT[lget(card, 1)] << 12)) for card in enterHand]
    playerCards, communityCards = simulatePokerSolitaire(numericCardHand, keyWord)
    playerWin, bestCommunityCombination = evaluateBetterHand(playerCards, communityCards)
    
    return playerWin, Card.print_pretty_cards(playerCards), Card.print_pretty_cards(communityCards), Card.print_pretty_cards(bestCommunityCombination)

In [10]:
enterHand = ['4d', '3s', 'As', '2d', '5c']

In [11]:
playPoker([], "FirstFive")

6340 4850


(False,
 ' [5♦],[6♣],[4♣],[K♣],[A♣] ',
 ' [7♦],[4♠],[8♦],[8♥],[5♠] ',
 ' [7♦],[4♠],[8♦],[8♥],[5♠] ')